In [542]:
import numpy as np
from random import *

In [543]:
# taler = 1
spielfeld = np.zeros([5,5])
spielfeld[0,:] = 2
spielfeld[4,:] = 2
spielfeld[:,0] = 2
spielfeld[:,4] = 2

spielfeld[3,3] = 1

spielfeld = spielfeld.astype(int)

In [544]:
state = [1,1]
learned = np.zeros([0,4])
epsilon = 0.1

In [545]:
spielfeld

array([[2, 2, 2, 2, 2],
       [2, 0, 0, 0, 2],
       [2, 0, 0, 0, 2],
       [2, 0, 0, 1, 2],
       [2, 2, 2, 2, 2]])

In [547]:
do_action(state, spielfeld, learned, epsilon)
# 1. Richtung des nächsten Coins: (1,2,3,4:links,oben,rechts,unten)
# 2.-5. Direkte Umgebung: (links, oben, rechts, unten; 0=frei, 1 = Coin, 2 = Wand)
# 3. 0. 0. 1. 2

learned:
[[ -1.8          0.           0.          -1.8       ]
 [ -1.8          0.           0.          -1.8       ]
 [  0.           0.          -1.8          0.        ]
 [ -1.8          0.           0.          -1.8       ]
 [  0.           0.           0.           0.        ]
 [  0.          -1.8          0.          -1.8       ]
 [  0.           0.          -1.8        180.        ]
 [  0.          -5.59998033  -2.00000398   0.        ]
 [  0.           0.          -1.8          0.        ]
 [ -2.00000002  -2.          -3.99997815   0.        ]
 [  0.          -4.00016356  -2.          -2.        ]
 [ -1.9999998   -2.          -2.00017998  -2.        ]
 [  0.          -2.00002178   0.          -2.18018   ]
 [ -3.99979602   0.          -2.18324128  -2.        ]
 [ -2.00019962  -2.           0.          -4.00163544]
 [ -2.00180004   0.          -2.           0.        ]
 [ -2.0000036    0.           0.          -2.00000002]]
observations:
[[3. 2. 2. 0. 0.]
 [3. 2. 0. 0. 0.]
 [3. 

In [535]:
def do_action(state, spielfeld, learned, epsilon):
    obs = np.zeros([0,5])
    for i in range(1000):
        # spielfeld[state[0], state[1]] = 2
        actions = np.array(["down", "up", "left", "right"])
        discount = 0.2
        learning_rate = 0.9
        n_state = [0,0]
        if False:
            print("+++++++++++++++++++")
        else:
            indices = np.array([])
            possible_actions = np.array([])
            current_obs = get_observation(spielfeld,state[0],state[1])
            # print(current_obs)
            obs, index_current, learned = update_and_get_obs(obs, current_obs, learned)

            if not state[0] == 1:
                #darf nach oben gehen
                new_obs = get_observation(spielfeld,state[0] - 1,state[1])
                # print(new_obs)
                obs, to_add, learned = update_and_get_obs(obs, new_obs, learned)
                indices = np.append(indices,to_add)
                possible_actions = np.append(possible_actions,["up"])
            if not state[0] == 3:
                # darf nach unten gehen
                new_obs = get_observation(spielfeld,state[0] + 1,state[1])
                obs, to_add, learned = update_and_get_obs(obs, new_obs, learned)
                indices = np.append(indices,to_add)
                possible_actions = np.append(possible_actions,["down"])
            if not state[1] == 3:
                # darf nach rechts gehen
                new_obs = get_observation(spielfeld,state[0],state[1] + 1)
                obs, to_add, learned = update_and_get_obs(obs, new_obs, learned)
                indices = np.append(indices,to_add)
                possible_actions = np.append(possible_actions,["right"])
            if not state[1] == 1:
                # darf nach links gehen
                new_obs = get_observation(spielfeld,state[0],state[1] - 1)
                obs, to_add, learned = update_and_get_obs(obs, new_obs, learned)
                indices = np.append(indices,to_add)
                possible_actions = np.append(possible_actions,["left"])

            indices = indices.astype(int)
            # Hole Werte der möglichen Folgezustände
            #print(obs)
            #print(indices)
            #print("x")
            values = learned[indices,:]
            #print(values)
            # print(learned)
           # optima = np.argmax(values.flatten(), axis = 0)
            if epsilon > uniform(0,1):
                # epsilon -= 0.001
                # Random Choice
                # print(indices)
                # print("RANDOM")
                b = values.flatten()
                # print(b)
                optima = np.random.choice(np.flatnonzero(b))
                best_choice = possible_actions[optima // 4]
                best_value = values.flatten()[optima]
            else:
                b = values.flatten()
                # print(b)
                optima = np.random.choice(np.flatnonzero(b == b.max()))
                best_choice = possible_actions[optima // 4]
                best_value = values.flatten()[optima]
           # print(best_choice, state)

            # get Reward
            if(best_choice == "down"):
                action_index = 0
                n_state[0] = state[0] + 1
                n_state[1] = state[1]
            elif(best_choice == "up"):
                action_index = 1
                n_state[0] = state[0] - 1
                n_state[1] = state[1]
            elif(best_choice == "left"):
                action_index = 2
                n_state[0] = state[0]
                n_state[1] = state[1] - 1
            elif(best_choice == "right"):
                action_index = 3
                n_state[0] = state[0]
                n_state[1] = state[1] + 1
                
            reward, spielfeld = get_reward(state,spielfeld,best_choice)
            learned[index_current, action_index] = (1-learning_rate) * learned[index_current,action_index] + learning_rate * (best_value + discount * reward)
            state = n_state
    print("learned:")
    print(learned)
    print("observations:")
    print(obs)

In [182]:
def get_reward(state,spielfeld,action):
    x,y = 0,0
    if action == "left":
        x = -1
    if action == "right":
        x = 1
    if action == "down":
        y = 1
    if action == "up":
        y = -1
        
    x +=state[1]    
    y +=state[0]
    if spielfeld[y,x] == 1:
        # collect coin
        spielfeld[y,x] = 0
        return 1000, spielfeld
    else:
        return -10, spielfeld

In [541]:
def get_observation(spielfeld, y, x):
    # Suche den Coin, der am nächsten ist (manhattan distance)
    coinsY, coinsX = np.where(spielfeld == 1)
    distancesY = np.abs(coinsY - y)
    distancesX = np.abs(coinsX - x)
    observation = np.array([0, 0,0,0,0]) 
    # 1. Richtung des nächsten Coins: (1,2,3,4:links,oben,rechts,unten)
    # 2. Direkte Umgebung: (links, oben, rechts, unten; 0=frei, 1 = Coin, 2 = Wand)
 
    if not distancesY.shape[0] == 0:
        distances = distancesY + distancesX
        coinY = coinsY[np.argmin(distances)]
        coinX = coinsX[np.argmin(distances)]
        # setze observation flags:
        if coinX < x:
            observation[0] = 1
        elif coinX > x:
            observation[0] = 3
        elif coinY > y:
            observation[0] = 2
        elif coinY < y:
            observation[0] = 4
    
    
    observation[1] = spielfeld[y, x - 1]
    observation[2] = spielfeld[y - 1, x]
    observation[3] = spielfeld[y, x + 1]
    observation[4] = spielfeld[y + 1, x]
    
    return observation

def update_and_get_obs(db, new_obs, learned):
    temp = -1
    # print(" ")
    # print("----")
    #print(db)
    #print(new_obs)
    #print("----")
    for i in range(db.shape[0]):
        #print(db[i])
        #print("==")
        #print(new_obs)
        if np.array_equal(db[i], new_obs):
            # print("old")
            return db, i, learned
    # print("new")
    db = np.append(db,np.array([new_obs]), axis = 0)
    learned = np.append(learned, np.zeros([1,learned.shape[1]]), axis = 0)
    # print(db)
    return db, (db.shape[0] - 1), learned

In [488]:
new = np.array([1,6,8])
#db = np.array([[1,3,2],[4,455,6]])
#learned = np.array([[2,3,6,6],[4,3,6,6]])

In [489]:
db, y, learned = update_and_get_obs(db, new,learned)

[[  1   3   2]
 [  4 455   6]
 [  2   6   8]]
[1 6 8]
new


In [490]:
db

array([[  1,   3,   2],
       [  4, 455,   6],
       [  2,   6,   8],
       [  1,   6,   8]])

In [491]:
learned

array([[2., 3., 6., 6.],
       [4., 3., 6., 6.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])